In [ ]:
# import etl libraries
import requests
import pandas as pd, numpy as np 
from bs4 import BeautifulSoup

In [10]:
# fetch data
iata_data = pd.read_csv("data/country-development-finance/csv/iata_data.csv")

In [52]:
MIN_YEAR = 1970
MAX_YEAR = 2024

years = list(range(MIN_YEAR, MAX_YEAR))
quarters = ["Q1", "Q2", "Q3", "Q4"]

dim_time = pd.DataFrame([(y, q) for y in years for q in quarters],
                        columns=["year", "quarter"])
dim_time["time_id"] = dim_time.index + 1

dim_time.head()

,year,quarter,time_id
0,1970,Q1,1
1,1970,Q2,2
2,1970,Q3,3
3,1970,Q4,4
4,1971,Q1,5


In [60]:
# filter data by year range listed in dim_time

iata_data_filtered = iata_data[
    (iata_data["Calendar Year"] >= MIN_YEAR) &
    (iata_data["Calendar Year"] <= MAX_YEAR)
]

iata_data_filtered.head()

,IATI Identifier,Title,Reporting Organisation Group,Reporting Organisation,Reporting Organisation Type,Aid Type,Finance Type,Flow Type,Provider Organisation,Provider Organisation Type,...,Calendar Year,Calendar Quarter,Calendar Year and Quarter,URL,Value (USD),Value (EUR),Value (Local currrency),country,iso_alpha2,country_id
36,41AAA-11295-001,Procurement and Supply Support of Anti-Tubercu...,UN - United Nations,United Nations Office for Project Services (UN...,40 - Multilateral,No data,No data,No data,United Nations Office for Project Services (UN...,No data,...,2016,Q1,2016 Q1,https://d-portal.org/q.html?aid=41AAA-11295-001,2986309.0,2.743005e+06,2.032183e+08,Afghanistan,AF,37
37,41AAA-11295-014,Diagnostics Procurement Support to Stop Tuberc...,UN - United Nations,United Nations Office for Project Services (UN...,40 - Multilateral,No data,No data,No data,United Nations Office for Project Services (UN...,No data,...,2018,Q1,2018 Q1,https://d-portal.org/q.html?aid=41AAA-11295-014,2895425.0,2.414262e+06,2.012126e+08,Afghanistan,AF,38
38,41AAA-11295-032,Support for the Stop Tuberculosis Partnership ...,UN - United Nations,United Nations Office for Project Services (UN...,40 - Multilateral,No data,No data,No data,United Nations Office for Project Services (UN...,No data,...,2021,Q2,2021 Q2,https://d-portal.org/q.html?aid=41AAA-11295-032,514886.0,4.220031e+05,3.998347e+07,Afghanistan,AF,39
39,41AAA-11960-007,Support to the Global Environment Facility (GE...,UN - United Nations,United Nations Office for Project Services (UN...,40 - Multilateral,No data,No data,No data,United Nations Office for Project Services (UN...,No data,...,2015,Q3,2015 Q3,https://d-portal.org/q.html?aid=41AAA-11960-007,1668446.0,1.491148e+06,1.013080e+08,Afghanistan,AF,40
40,41AAA-20431-001,Project to Support Improvement of Agricultural...,UN - United Nations,United Nations Office for Project Services (UN...,40 - Multilateral,No data,No data,No data,United Nations Office for Project Services (UN...,No data,...,2017,Q1,2017 Q1,https://d-portal.org/q.html?aid=41AAA-20431-001,9475434.0,8.941619e+06,6.372229e+08,Afghanistan,AF,41


In [58]:
# see countries column
countries = iata_data_filtered["Recipient Country or Region"].unique()
country_names = pd.Series(countries)

country_names

0                                      AF - Afghanistan
1                                       BD - Bangladesh
2                 BO - Bolivia (Plurinational State of)
3                                         CO - Colombia
4                                            EG - Egypt
5                                         ET - Ethiopia
6     GB - United Kingdom of Great Britain and North...
7                                          GE - Georgia
8                                            GH - Ghana
9                                        GT - Guatemala
10                                        HN - Honduras
11                                           HT - Haiti
12                                          JO - Jordan
13                                           KE - Kenya
14                                        KH - Cambodia
15                                         LB - Lebanon
16                       MD - Moldova (the Republic of)
17                                         NG - 

In [62]:
# convert countries to simplier country names
country_map = {
    "AF - Afghanistan": "Afghanistan",
    "BD - Bangladesh": "Bangladesh",
    "BO - Bolivia (Plurinational State of)": "Bolivia",
    "CO - Colombia": "Colombia",
    "EG - Egypt": "Egypt",
    "ET - Ethiopia": "Ethiopia",
    "GB - United Kingdom of Great Britain and Northern Ireland (the)": "United Kingdom",
    "GE - Georgia": "Georgia",
    "GH - Ghana": "Ghana",
    "GT - Guatemala": "Guatemala",
    "HN - Honduras": "Honduras",
    "HT - Haiti": "Haiti",
    "JO - Jordan": "Jordan",
    "KE - Kenya": "Kenya",
    "KH - Cambodia": "Cambodia",
    "LB - Lebanon": "Lebanon",
    "MD - Moldova (the Republic of)": "Moldova",
    "NG - Nigeria": "Nigeria",
    "NI - Nicaragua": "Nicaragua",
    "NP - Nepal": "Nepal",
    "PH - Philippines (the)": "Philippines",
    "PK - Pakistan": "Pakistan",
    "RW - Rwanda": "Rwanda",
    "SL - Sierra Leone": "Sierra Leone",
    "SN - Senegal": "Senegal",
    "TZ - Tanzania, the United Republic of": "Tanzania",
    "UA - Ukraine": "Ukraine",
    "UG - Uganda": "Uganda",
    "US - United States of America (the)": "United States",
    "VN - Viet Nam": "Vietnam",
    "YE - Yemen": "Yemen"
}
# get iso alpha id
iata_data_filtered.loc[:, "country"] = (
    iata_data_filtered["Recipient Country or Region"]
    .map(country_map)
)

# map countries to simpler form
iata_data_filtered.loc[:, "iso_alpha2"] = (
    iata_data_filtered["Recipient Country or Region"]
    .str.split(" - ", expand=True)[0]
)

In [63]:
iata_data["iso_alpha2"].unique()

array(['AF', 'BD', 'BO', 'CO', 'EG', 'ET', 'GB', 'GE', 'GH', 'GT', 'HN',
       'HT', 'JO', 'KE', 'KH', 'LB', 'MD', 'NG', 'NI', 'NP', 'PH', 'PK',
       'RW', 'SL', 'SN', 'TZ', 'UA', 'UG', 'US', 'VN', 'YE'], dtype=object)

In [64]:
iata_data["country"].unique()

array(['Afghanistan', 'Bangladesh', 'Bolivia', 'Colombia', 'Egypt',
       'Ethiopia', 'United Kingdom', 'Georgia', 'Ghana', 'Guatemala',
       'Honduras', 'Haiti', 'Jordan', 'Kenya', 'Cambodia', 'Lebanon',
       'Moldova', 'Nigeria', 'Nicaragua', 'Nepal', 'Philippines',
       'Pakistan', 'Rwanda', 'Sierra Leone', 'Senegal', 'Tanzania',
       'Ukraine', 'Uganda', 'United States', 'Vietnam', 'Yemen'],
      dtype=object)

In [223]:
# dim_country table
dim_country = iata_data[["country_id", "country", "iso_alpha2"]]
dim_country.head()


,country_id,country,iso_alpha2
0,1,Afghanistan,AF
1,2,Afghanistan,AF
2,3,Afghanistan,AF
3,4,Afghanistan,AF
4,5,Afghanistan,AF


In [70]:
# extract organizations, types, roles via pivot longer
organizations_long = iata_data.melt(
    id_vars = ["IATI Identifier"],
    value_vars = [
        "Reporting Organisation",
        "Provider Organisation",
        "Receiver Organisation" 
    ],
    var_name = "organization_role",
    value_name = "organization_name"
)

organization_types_long = iata_data.melt(
    id_vars = ["IATI Identifier"],
    value_vars = [
        "Reporting Organisation Type",
        "Provider Organisation Type",
        "Receiver Organisation Type" 
    ],
    var_name = "role_type",
    value_name = "organization_type"
)

In [72]:
organizations_long.head()

,IATI Identifier,organization_role,organization_name
0,41120-100879,Reporting Organisation,UN-Habitat [41120]
1,41120-100879,Reporting Organisation,UN-Habitat [41120]
2,41120-102631,Reporting Organisation,UN-Habitat [41120]
3,41120-102631,Reporting Organisation,UN-Habitat [41120]
4,41120-102645,Reporting Organisation,UN-Habitat [41120]


In [73]:
organization_types_long.head()

,IATI Identifier,role_type,organization_type
0,41120-100879,Reporting Organisation Type,40 - Multilateral
1,41120-100879,Reporting Organisation Type,40 - Multilateral
2,41120-102631,Reporting Organisation Type,40 - Multilateral
3,41120-102631,Reporting Organisation Type,40 - Multilateral
4,41120-102645,Reporting Organisation Type,40 - Multilateral


In [95]:
organizations_combined = pd.concat([organizations_long, organization_types_long["organization_type"]], axis=1)
organizations_combined = organizations_combined.dropna(subset=["organization_name"])
organizations_combined["organization_role"] = organizations_combined["organization_role"].str.split().str[0]

organizations_combined.head()

,IATI Identifier,organization_role,organization_name,organization_type
0,41120-100879,Reporting,UN-Habitat [41120],40 - Multilateral
1,41120-100879,Reporting,UN-Habitat [41120],40 - Multilateral
2,41120-102631,Reporting,UN-Habitat [41120],40 - Multilateral
3,41120-102631,Reporting,UN-Habitat [41120],40 - Multilateral
4,41120-102645,Reporting,UN-Habitat [41120],40 - Multilateral


In [99]:
# extract ids from org name and type, store in seperate columns

organizations_combined["organization_iati_id"] = (
    organizations_combined["organization_name"]
    .str.extract(r"\[(\d+)\]")
)

organizations_combined["organization_name_clean"] = (
    organizations_combined["organization_name"]
    .str.replace(r"\s*\[\d+\]", "", regex=True)
)

organizations_combined["organization_type_code"] = (
    organizations_combined["organization_type"]
    .str.extract(r"^(\d+)")
)

organizations_combined["organization_type_name"] = (
    organizations_combined["organization_type"]
    .str.replace(r"^\d+\s*-\s*", "", regex=True)
)

organizations_combined.head()

,IATI Identifier,organization_role,organization_name,organization_type,organization_iati_id,organization_name_clean,organization_type_code,organization_type_name
0,41120-100879,Reporting,UN-Habitat [41120],40 - Multilateral,41120,UN-Habitat,40,Multilateral
1,41120-100879,Reporting,UN-Habitat [41120],40 - Multilateral,41120,UN-Habitat,40,Multilateral
2,41120-102631,Reporting,UN-Habitat [41120],40 - Multilateral,41120,UN-Habitat,40,Multilateral
3,41120-102631,Reporting,UN-Habitat [41120],40 - Multilateral,41120,UN-Habitat,40,Multilateral
4,41120-102645,Reporting,UN-Habitat [41120],40 - Multilateral,41120,UN-Habitat,40,Multilateral


In [102]:
# deduplicate organiztions

dim_organization = (
    organizations_combined[
        [
            "organization_name_clean",
            "organization_iati_id",
            "organization_type_code",
            "organization_type_name"
        ]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)


# rename column
dim_organization = dim_organization.rename(
    columns={"organization_name_clean": "organization_name"}
)


# add pk
dim_organization["organization_id"] = dim_organization.index + 1

# rearrange org columns
dim_organization = dim_organization[
    [
        "organization_id",
        "organization_name",
        "organization_iati_id",
        "organization_type_code",
        "organization_type_name"
    ]
]

dim_organization.head()

,organization_id,organization_name,organization_iati_id,organization_type_code,organization_type_name
0,1,UN-Habitat,41120,40,Multilateral
1,2,United Nations Office for Project Services (UN...,NaN,40,Multilateral
2,3,"The Global Fund to Fight AIDS, Tuberculosis an...",47045,30,Public Private Partnership
3,4,"Gavi, the vaccine alliance",47122,40,Multilateral
4,5,Swedish Committee for Afghanistan [AF-MOE-118],NaN,21,International NGO


In [103]:
# add null row because there are instances where organization column can be null

unknown_row = pd.DataFrame([{
    "organization_id": 0,
    "organization_name": "Unknown",
    "organization_iati_id": None,
    "organization_type_code": None,
    "organization_type_name": "Unknown"
}])

dim_organization = pd.concat([unknown_row, dim_organization], ignore_index=True)

In [104]:
# get sector fields for dim sector

sector_staging = iata_data[["Sector Category", "Sector"]].copy()
sector_staging = sector_staging.dropna().drop_duplicates()

sector_staging.head()

,Sector Category,Sector
0,160 - Other Social Infrastructure & Services,16030 - Housing policy and administrative mana...
2,730 - Reconstruction Relief & Rehabilitation,73010 - Immediate post-emergency reconstructio...
4,430 - Other Multisector,43030 - Urban development and management
16,430 - Other Multisector,43082 - Research/scientific institutions
36,120 - Health,12263 - Tuberculosis control


In [105]:
# split sector category into code + name
sector_staging["sector_category_code"] = (
    sector_staging["Sector Category"].str.extract(r"^(\d+)")
)

sector_staging["sector_category"] = (
    sector_staging["Sector Category"].str.replace(r"^\d+\s*-\s*", "", regex=True)
)


In [107]:
# split sector into code + name
sector_staging["sector_code"] = (sector_staging["Sector"].str.extract(r"^(\d+)"))

sector_staging["sector_name"] = (sector_staging["Sector"].str.replace(r"^\d+\s*-\s*", "", regex=True))


In [109]:
# set up sector dimension

dim_sector = sector_staging[
    [
        "sector_code",
        "sector_name",
        "sector_category_code",
        "sector_category"
    ]
].drop_duplicates().reset_index(drop=True)

# add primary key
dim_sector["sector_id"] = dim_sector.index + 1

# reorder cols
dim_sector = dim_sector[
    [
        "sector_id",
        "sector_code",
        "sector_name",
        "sector_category_code",
        "sector_category"
    ]
]

dim_sector.head()

,sector_id,sector_code,sector_name,sector_category_code,sector_category
0,1,16030,Housing policy and administrative management,160,Other Social Infrastructure & Services
1,2,73010,Immediate post-emergency reconstruction and re...,730,Reconstruction Relief & Rehabilitation
2,3,43030,Urban development and management,430,Other Multisector
3,4,43082,Research/scientific institutions,430,Other Multisector
4,5,12263,Tuberculosis control,120,Health


In [133]:
aid_staging = iata_data[["Aid Type"]].dropna().drop_duplicates()
aid_staging.head()

,Aid Type
0,No data
69,C01 - Project-type interventions
1181,B03 - Contributions to specific-purpose progra...
1775,E01 - Scholarships/training in donor country
1956,B04 - Basket funds/pooled funding


In [160]:
# split aid type into code and name


aid_staging["aid_type_code"] = aid_staging["Aid Type"].str.extract(r"^(\S+)")
aid_staging["aid_type_name"] = aid_staging["Aid Type"].str.replace(r"^\S+\s*-\s*", "", regex=True)


# set type code to none for int
aid_staging["aid_type_code"] = aid_staging["aid_type_code"].replace("No", None)
aid_staging.head()

,Aid Type,aid_type_code,aid_type_name
0,No data,None,No data
69,C01 - Project-type interventions,C01,Project-type interventions
1181,B03 - Contributions to specific-purpose progra...,B03,Contributions to specific-purpose programmes a...
1775,E01 - Scholarships/training in donor country,E01,Scholarships/training in donor country
1956,B04 - Basket funds/pooled funding,B04,Basket funds/pooled funding


In [161]:
# build aid dimension

dim_aid_type = (
    aid_staging[["aid_type_code", "aid_type_name"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

# add pk
dim_aid_type["aid_type_id"] = dim_aid_type.index + 1

dim_aid_type = dim_aid_type[
    ["aid_type_id", "aid_type_code", "aid_type_name"]
]

dim_aid_type.head()


,aid_type_id,aid_type_code,aid_type_name
0,1,None,No data
1,2,C01,Project-type interventions
2,3,B03,Contributions to specific-purpose programmes a...
3,4,E01,Scholarships/training in donor country
4,5,B04,Basket funds/pooled funding


In [137]:
# flow type staging
flow_staging = iata_data[["Flow Type"]].copy()
flow_staging = flow_staging.drop_duplicates()


flow_staging.head()

,Flow Type
0,No data
69,10 - ODA
2531,21 - Non-export credit OOF
4421,30 - Private Development Finance
4427,50 - Other flows


In [159]:
# split flow type into code and name
flow_staging["flow_type_code"] = flow_staging["Flow Type"].str.extract(r"^(\S+)")

flow_staging["flow_type_name"] = flow_staging["Flow Type"].str.replace(
    r"^\S+\s*-\s*", "", regex=True
)

# set type code to none as well to support int
flow_staging["flow_type_code"] = flow_staging["flow_type_code"].replace("No", None)

flow_staging.head()

,Flow Type,flow_type_code,flow_type_name
0,No data,None,No data
69,10 - ODA,10,ODA
2531,21 - Non-export credit OOF,21,Non-export credit OOF
4421,30 - Private Development Finance,30,Private Development Finance
4427,50 - Other flows,50,Other flows


In [145]:
# build flow dimension

dim_flow_type = (
    flow_staging[["flow_type_code", "flow_type_name"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

# add pk
dim_flow_type["flow_type_id"] = dim_flow_type.index + 1

dim_flow_type = dim_flow_type[
    ["flow_type_id", "flow_type_code", "flow_type_name"]
]


dim_flow_type.head()


,flow_type_id,flow_type_code,flow_type_name
0,1,No Data,No data
1,2,10,ODA
2,3,21,Non-export credit OOF
3,4,30,Private Development Finance
4,5,50,Other flows


In [148]:
transaction_staging = iata_data[["Transaction Type"]].copy()
transaction_staging = transaction_staging.drop_duplicates()


transaction_staging.head()

,Transaction Type
0,4 - Expenditure
1,1 - Incoming Funds
69,2 - Outgoing Commitment
72,3 - Disbursement
73955,budget - Budget


In [150]:
transaction_staging["transaction_type_code"] = transaction_staging["Transaction Type"].str.extract(r"^(\S+)")
transaction_staging["transaction_type_name"] = transaction_staging["Transaction Type"].str.replace(r"^\S+\s*-\s*", "", regex=True)


transaction_staging.head()

,Transaction Type,transaction_type_code,transaction_type_name
0,4 - Expenditure,4,Expenditure
1,1 - Incoming Funds,1,Incoming Funds
69,2 - Outgoing Commitment,2,Outgoing Commitment
72,3 - Disbursement,3,Disbursement
73955,budget - Budget,budget,Budget


In [153]:
# build transaction dimension
dim_transaction_type = (
    transaction_staging[["transaction_type_code", "transaction_type_name"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
# pk
dim_transaction_type["transaction_type_id"] = dim_transaction_type.index + 1

# reorder cols
dim_transaction_type = dim_transaction_type[
    ["transaction_type_id", "transaction_type_code", "transaction_type_name"]
]

dim_transaction_type.head()



,transaction_type_id,transaction_type_code,transaction_type_name
0,1,4,Expenditure
1,2,1,Incoming Funds
2,3,2,Outgoing Commitment
3,4,3,Disbursement
4,5,budget,Budget


In [155]:
# finance type staging

finance_staging = iata_data[["Finance Type"]].copy()
finance_staging = finance_staging.drop_duplicates()

finance_staging.head()

,Finance Type
0,No data
69,110 - Standard grant
6078,311 - Capital subscription on encashment basis
8969,421 - Standard loan
12165,511 - Acquisition of equity not part of joint ...


In [162]:
# extract code and name from finance type

finance_staging["finance_type_code"] = finance_staging["Finance Type"].str.extract(r"^(\S+)")
finance_staging["finance_type_name"] = finance_staging["Finance Type"].str.replace(r"^\S+\s*-\s*", "", regex=True)

# set type code to no data as well for better null consistency
finance_staging["finance_type_code"] = finance_staging["finance_type_code"].replace("No", None)


finance_staging.head()

,Finance Type,finance_type_code,finance_type_name
0,No data,None,No data
69,110 - Standard grant,110,Standard grant
6078,311 - Capital subscription on encashment basis,311,Capital subscription on encashment basis
8969,421 - Standard loan,421,Standard loan
12165,511 - Acquisition of equity not part of joint ...,511,Acquisition of equity not part of joint ventur...


In [164]:
# build finance dimension

dim_finance_type = (
    finance_staging[["finance_type_code", "finance_type_name"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

dim_finance_type["finance_type_id"] = dim_finance_type.index + 1


dim_finance_type = dim_finance_type[
    ["finance_type_id", "finance_type_code", "finance_type_name"]
]

dim_finance_type.head()

,finance_type_id,finance_type_code,finance_type_name
0,1,None,No data
1,2,110,Standard grant
2,3,311,Capital subscription on encashment basis
3,4,421,Standard loan
4,5,511,Acquisition of equity not part of joint ventur...


In [167]:
world_indicator_data = pd.read_csv("data/world-development-indicators/data.csv")

world_indicator_data.head()

,Country Name,Country Code,Series Name,Series Code,1970 [YR1970],1971 [YR1971],1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],...,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023],2024 [YR2024]
0,Afghanistan,AFG,"Population, total",SP.POP.TOTL,11290128,11567667,11853696,12157999,12469127,12773954,...,33831764,34700612,35688935,36743039,37856121,39068979,40000412,40578842,41454761,42647492
1,Afghanistan,AFG,"Lower secondary completion rate, total (% of r...",SE.SEC.CMPT.LO.ZS,..,..,..,9.78837013244629,8.57250022888184,..,...,..,52.3230895996094,54.7716407775879,57.0390014648438,60.4455986022949,..,..,..,..,..
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,..,..,..,..,..,..,...,565.569730408751,522.082215583898,525.469770891619,491.337221382603,496.6025042585,510.787063366811,356.496214115892,357.261152798144,413.757894705303,..
3,Afghanistan,AFG,Net ODA received per capita (current US$),DT.ODA.ODAT.PC.ZS,2.44549934335125,3.84174255938635,4.65508819402622,4.5829911008138,3.92248790532105,5.24348242971077,...,126.337665225459,117.272119196072,106.804449629749,103.216537364057,109.298305108472,107.766175643469,117.291553127472,96.4780375205293,73.8233974820921,..
4,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,37.46,37.932,38.423,38.951,39.469,39.994,...,62.27,62.646,62.406,62.443,62.941,61.454,60.417,65.617,66.035,..


In [179]:
# long pivot the years 

indicator_long = world_indicator_data.melt(
    id_vars=["Country Name", "Country Code", "Series Name", "Series Code"],
    var_name="year",
    value_name="value"
)

indicator_long.head()

,Country Name,Country Code,Series Name,Series Code,year,value
0,Afghanistan,AFG,"Population, total",SP.POP.TOTL,1970 [YR1970],11290128
1,Afghanistan,AFG,"Lower secondary completion rate, total (% of r...",SE.SEC.CMPT.LO.ZS,1970 [YR1970],..
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,1970 [YR1970],..
3,Afghanistan,AFG,Net ODA received per capita (current US$),DT.ODA.ODAT.PC.ZS,1970 [YR1970],2.44549934335125
4,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1970 [YR1970],37.46


In [180]:
# year has weird values so convert

indicator_long["year"] = indicator_long["year"].str.extract(r"(\d{4})").astype(int)

indicator_long.head(1)

,Country Name,Country Code,Series Name,Series Code,year,value
0,Afghanistan,AFG,"Population, total",SP.POP.TOTL,1970,11290128


In [181]:
# look at distinct 

indicator_long["Series Name"].unique()

array(['Population, total',
       'Lower secondary completion rate, total (% of relevant age group)',
       'GDP per capita (current US$)',
       'Net ODA received per capita (current US$)',
       'Life expectancy at birth, total (years)',
       'Mortality rate, under-5 (per 1,000 live births)',
       'Primary completion rate, total (% of relevant age group)',
       'School enrollment, primary (% net)', nan], dtype=object)

In [182]:
# make a map to convert the names to readable format

indicator_map = {
    "Population, total": "population",
    "GDP per capita (current US$)": "gdp_per_capita",
    "Net ODA received per capita (current US$)": "net_oda_per_capita",
    "Life expectancy at birth, total (years)": "life_expectancy",
    "Mortality rate, under-5 (per 1,000 live births)": "under5_mortality",
    "Primary completion rate, total (% of relevant age group)": "primary_completion_rate",
    "Lower secondary completion rate, total (% of relevant age group)": "lower_secondary_completion_rate",
    "School enrollment, primary (% net)": "school_enrollment_primary"
}

indicator_long["metric"] = long_df["Series Name"].map(indicator_map)

indicator_long.head()

,Country Name,Country Code,Series Name,Series Code,year,value,metric
0,Afghanistan,AFG,"Population, total",SP.POP.TOTL,1970,11290128,population
1,Afghanistan,AFG,"Lower secondary completion rate, total (% of r...",SE.SEC.CMPT.LO.ZS,1970,..,lower_secondary_completion_rate
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,1970,..,gdp_per_capita
3,Afghanistan,AFG,Net ODA received per capita (current US$),DT.ODA.ODAT.PC.ZS,1970,2.44549934335125,net_oda_per_capita
4,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1970,37.46,life_expectancy


In [187]:
# theres weird rows where theres no data at all so filter it out (verified using R)

indicator_long = indicator_long[indicator_long["metric"].notna()]

# theres "..." entries as opposed to n/a entries
indicator_long["value"] = indicator_long["value"].replace("..", None)

# convert numeric
indicator_long["value"] = pd.to_numeric(indicator_long["value"], errors="coerce")



In [188]:
# pivot again to convert metrics to seperate columns

indicator_pivoted = indicator_long.pivot_table(
    index=["Country Name", "Country Code", "year"],
    columns="metric",
    values="value"
).reset_index()

indicator_pivoted.head()

metric,Country Name,Country Code,year,gdp_per_capita,life_expectancy,lower_secondary_completion_rate,net_oda_per_capita,population,primary_completion_rate,school_enrollment_primary,under5_mortality
0,Afghanistan,AFG,1970,NaN,37.460,NaN,2.445499,11290128.0,NaN,NaN,301.6
1,Afghanistan,AFG,1971,NaN,37.932,NaN,3.841743,11567667.0,NaN,NaN,296.5
2,Afghanistan,AFG,1972,NaN,38.423,NaN,4.655088,11853696.0,NaN,NaN,291.4
3,Afghanistan,AFG,1973,NaN,38.951,9.78837,4.582991,12157999.0,NaN,NaN,285.9
4,Afghanistan,AFG,1974,NaN,39.469,8.57250,3.922488,12469127.0,17.058519,26.82221,280.6


In [189]:
# verify country names
indicator_pivoted["Country Name"].unique()

array(['Afghanistan', 'Bangladesh', 'Bolivia', 'Cambodia', 'Colombia',
       'Egypt, Arab Rep.', 'Ethiopia', 'Georgia', 'Guatemala', 'Haiti',
       'Honduras', 'Indonesia', 'Jordan', 'Kenya', 'Lebanon',
       'Madagascar', 'Moldova', 'Nepal', 'Nicaragua', 'Nigeria',
       'Pakistan', 'Philippines', 'Rwanda', 'Senegal', 'Sierra Leone',
       'Tanzania', 'Uganda', 'Ukraine', 'United Kingdom', 'United States',
       'Viet Nam', 'Yemen, Rep.'], dtype=object)

In [224]:
# map country names to 

country_mapping = {
    "Afghanistan": "Afghanistan",
    "Bangladesh": "Bangladesh",
    "Bolivia": "Bolivia",
    "Cambodia": "Cambodia",
    "Colombia": "Colombia",
    "Egypt, Arab Rep.": "Egypt",
    "Ethiopia": "Ethiopia",
    "Georgia": "Georgia",
    "Guatemala": "Guatemala",
    "Haiti": "Haiti",
    "Honduras": "Honduras",
    "Indonesia": "Indonesia",
    "Jordan": "Jordan",
    "Kenya": "Kenya",
    "Lebanon": "Lebanon",
    "Madagascar": "Madagascar",
    "Moldova": "Moldova",
    "Nepal": "Nepal",
    "Nicaragua": "Nicaragua",
    "Nigeria": "Nigeria",
    "Pakistan": "Pakistan",
    "Philippines": "Philippines",
    "Rwanda": "Rwanda",
    "Senegal": "Senegal",
    "Sierra Leone": "Sierra Leone",
    "Tanzania": "Tanzania",
    "Uganda": "Uganda",
    "Ukraine": "Ukraine",
    "United Kingdom": "United Kingdom",
    "United States": "United States",
    "Viet Nam": "Vietnam",
    "Yemen, Rep.": "Yemen"
}

indicator_pivoted["country_clean"] = indicator_pivoted["Country Name"].map(country_mapping)

indicator_pivoted.head()

metric,Country Name,Country Code,year,gdp_per_capita,life_expectancy,lower_secondary_completion_rate,net_oda_per_capita,population,primary_completion_rate,school_enrollment_primary,under5_mortality,country_clean,quarter
0,Afghanistan,AFG,1970,NaN,37.460,NaN,2.445499,11290128.0,NaN,NaN,301.6,Afghanistan,Q1
1,Afghanistan,AFG,1971,NaN,37.932,NaN,3.841743,11567667.0,NaN,NaN,296.5,Afghanistan,Q1
2,Afghanistan,AFG,1972,NaN,38.423,NaN,4.655088,11853696.0,NaN,NaN,291.4,Afghanistan,Q1
3,Afghanistan,AFG,1973,NaN,38.951,9.78837,4.582991,12157999.0,NaN,NaN,285.9,Afghanistan,Q1
4,Afghanistan,AFG,1974,NaN,39.469,8.57250,3.922488,12469127.0,17.058519,26.82221,280.6,Afghanistan,Q1


In [225]:
# i realized my dim country was duplicated

country_staging = iata_data[["country", "iso_alpha2"]].copy()
country_staging = country_staging.drop_duplicates().reset_index(drop=True)

country_staging.head()

,country,iso_alpha2
0,Afghanistan,AF
1,Bangladesh,BD
2,Bolivia,BO
3,Colombia,CO
4,Egypt,EG


In [226]:
# pk
country_staging["country_id"] = country_staging.index + 1
# recreate dimcountry
dim_country = country_staging[["country_id", "country", "iso_alpha2"]]

dim_country.head()

,country_id,country,iso_alpha2
0,1,Afghanistan,AF
1,2,Bangladesh,BD
2,3,Bolivia,BO
3,4,Colombia,CO
4,5,Egypt,EG


In [227]:
next_id = dim_country["country_id"].max() + 1

# manually add indonesia cause missing country
indonesia_row = pd.DataFrame([{
    "country_id": next_id,
    "country": "Indonesia",
    "iso_alpha2": "ID"  
}])

dim_country = pd.concat([dim_country, indonesia_row], ignore_index=True)

# make sure indonesia exists
"Indonesia" in dim_country["country"].unique()

True

In [228]:
# add quarter for joining

indicator_pivoted["quarter"] = "Q1"


In [229]:
fact_indicator = indicator_pivoted.merge(
    dim_country[["country_id", "country"]],
    left_on="country_clean",
    right_on="country",
    how="left"
)


fact_indicator.head()

,Country Name,Country Code,year,gdp_per_capita,life_expectancy,lower_secondary_completion_rate,net_oda_per_capita,population,primary_completion_rate,school_enrollment_primary,under5_mortality,country_clean,quarter,country_id,country
0,Afghanistan,AFG,1970,NaN,37.460,NaN,2.445499,11290128.0,NaN,NaN,301.6,Afghanistan,Q1,1.0,Afghanistan
1,Afghanistan,AFG,1971,NaN,37.932,NaN,3.841743,11567667.0,NaN,NaN,296.5,Afghanistan,Q1,1.0,Afghanistan
2,Afghanistan,AFG,1972,NaN,38.423,NaN,4.655088,11853696.0,NaN,NaN,291.4,Afghanistan,Q1,1.0,Afghanistan
3,Afghanistan,AFG,1973,NaN,38.951,9.78837,4.582991,12157999.0,NaN,NaN,285.9,Afghanistan,Q1,1.0,Afghanistan
4,Afghanistan,AFG,1974,NaN,39.469,8.57250,3.922488,12469127.0,17.058519,26.82221,280.6,Afghanistan,Q1,1.0,Afghanistan


In [230]:
dim_country

,country_id,country,iso_alpha2
0,1,Afghanistan,AF
1,2,Bangladesh,BD
2,3,Bolivia,BO
3,4,Colombia,CO
4,5,Egypt,EG
5,6,Ethiopia,ET
6,7,United Kingdom,GB
7,8,Georgia,GE
8,9,Ghana,GH
9,10,Guatemala,GT
